# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""7366076767064011405""",3.7408e18,"""../../../../data/SampleHDS.jso…","""11445862371370455124""","""996186440912158919""","""5355853042024550197""",false
"""13885531647443271493""",1.7482e19,"""../../../../data/SampleHDS.jso…","""4656569416167498050""","""996186440912158919""","""7580775243754422822""",true
"""7067119178935877299""",8.8094e18,"""../../../../data/SampleHDS.jso…","""14347826003968334750""","""17121075345685997428""","""5355853042024550197""",false
"""13885531647443271493""",8.2588e18,"""../../../../data/SampleHDS.jso…","""4078697061539732141""","""8830867801777472262""","""5355853042024550197""",false
"""7366076767064011405""",3.8918e18,"""../../../../data/SampleHDS.jso…","""13288364406532973451""","""8830867801777472262""","""11508028355282886263""",true
"""7067119178935877299""",3.5203e18,"""../../../../data/SampleHDS.jso…","""338682199015692088""","""17955181812743147096""",null,false
"""7067119178935877299""",1.2742e19,"""../../../../data/SampleHDS.jso…","""3778157295174976606""","""8830867801777472262""","""11508028355282886263""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""5025318156029100158""",6.5138e18,"""../../../../data/SampleHDS.jso…","""8855968667078076054""","""FirstMortgage30yr""","""10299141435995342729""","""Rejected"""
"""3870675745504902947""",1.2726e19,"""../../../../data/SampleHDS.jso…","""15270368382607087193""","""FirstMortgage30yr""","""3131593840207048690""","""Accepted"""
"""12362868346653527697""",6.4425e18,"""../../../../data/SampleHDS.jso…","""5026172220578793301""","""MoneyMarketSavingsAccount""","""10299141435995342729""","""Rejected"""
"""3870675745504902947""",3.3595e18,"""../../../../data/SampleHDS.jso…","""761022111288795900""","""BasicChecking""","""10299141435995342729""","""Rejected"""
"""5025318156029100158""",1.3084e19,"""../../../../data/SampleHDS.jso…","""6853274655965317797""","""BasicChecking""","""13546059814940573562""","""Accepted"""
"""12362868346653527697""",2.8984e18,"""../../../../data/SampleHDS.jso…","""6481845371923793215""","""UPlusFinPersonal""",null,"""Rejected"""
"""12362868346653527697""",5.8330e18,"""../../../../data/SampleHDS.jso…","""4888913171151449595""","""BasicChecking""","""13546059814940573562""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""17137216660075055480""",1.5534e19,"""../../../../data/SampleHDS.jso…","""15256477228950604321""","""FirstMortgage30yr""","""5365765740189946508""","""Rejected"""
"""9239508125333686845""",1.5585e19,"""../../../../data/SampleHDS.jso…","""5748329447039337521""","""FirstMortgage30yr""","""13945789067113756315""","""Accepted"""
"""1195875609871060452""",3.6761e18,"""../../../../data/SampleHDS.jso…","""7722774600078556983""","""MoneyMarketSavingsAccount""","""5365765740189946508""","""Rejected"""
"""9239508125333686845""",1.2428e19,"""../../../../data/SampleHDS.jso…","""16787501570511074124""","""BasicChecking""","""5365765740189946508""","""Rejected"""
"""17137216660075055480""",1.3841e19,"""../../../../data/SampleHDS.jso…","""7869921592204567871""","""BasicChecking""","""4458569829142509689""","""Accepted"""
"""1195875609871060452""",8.4453e18,"""../../../../data/SampleHDS.jso…","""17365505558168836504""","""UPlusFinPersonal""",null,"""Rejected"""
"""1195875609871060452""",2.5789e18,"""../../../../data/SampleHDS.jso…","""4983691918605320341""","""BasicChecking""","""4458569829142509689""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""15650802729957068662""",7.5595e18,"""6465019140683639660""","""FirstMortgage30yr""","""1336767213852904941""","""Rejected"""
"""7597142121382374978""",1.6323e19,"""11330766791358935844""","""FirstMortgage30yr""","""11107915217502457470""","""Accepted"""
"""4826529943883689700""",1.1296e19,"""10160209909570538756""","""MoneyMarketSavingsAccount""","""1336767213852904941""","""Rejected"""
"""7597142121382374978""",1.5711e18,"""10137487717787294801""","""BasicChecking""","""1336767213852904941""","""Rejected"""
"""15650802729957068662""",1.2622e19,"""2147067015790250544""","""BasicChecking""","""5168795359902163199""","""Accepted"""
"""4826529943883689700""",1.7312e18,"""2849707428096218412""","""UPlusFinPersonal""",null,"""Rejected"""
"""4826529943883689700""",1.7401e19,"""10047033351636202882""","""BasicChecking""","""5168795359902163199""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'